In [1]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erfc
import ipywidgets as widgets
from ipywidgets import RadioButtons
from IPython.display import display, clear_output
from scipy.signal import upfirdn, convolve
from ipywidgets import Checkbox, Button, Output, VBox, HBox, Dropdown, Layout
import time
print("Libraries added successfully!")

Libraries added successfully!


In [2]:
# Loading animation
loading = """
    <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
        <div class='loader' style='border: 12px solid #f3f3f3; /* Light grey */
                                     border-top: 12px solid #01cc97; /* Blue */
                                     border-radius: 50%;
                                     width: 40px;
                                     height: 40px;
                                     animation: spin 2s linear infinite;'></div>
    </div>
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
done = """
        <div style='display: flex; justify-content: center; align-items: center; height: 80px;'>
            <div style='font-size: 40px; color: #01cc97;'>&#10003;</div>
        </div>
        """
loader_html3 = widgets.HTML(
  value=loading
)
timer_html3 = widgets.HTML(
    value="Elapsed time: - seconds"
)

# Define the modulation schemes and levels
modulation_schemes = {
    'ASK': ['2-ASK', '4-ASK', '8-ASK', '16-ASK', '32-ASK', '64-ASK'],
    'QAM': ['4-QAM', '16-QAM', '64-QAM', '256-QAM'],
    'PSK': ['4-PSK', '8-PSK', '16-PSK', '32-PSK', '64-PSK'],
    'FSK': ['4-FSK', '8-FSK', '16-FSK', '32-FSK', '64-FSK'],
    'MSK': ['MSK (with precoding)', 'MSK (without precoding)']
}

# Dropdown for modulation schemes
modulation_dropdown = widgets.Dropdown(
    options=list(modulation_schemes.keys()),
    value='ASK',
    description='Modulation:'
)

# Dropdown for levels
levels_dropdown = widgets.Dropdown(
    description='Levels:'
)

def update_levels_dropdown(*args):
    levels_dropdown.options = modulation_schemes[modulation_dropdown.value]
    levels_dropdown.value = levels_dropdown.options[0]  # Set default value

modulation_dropdown.observe(update_levels_dropdown, 'value')
update_levels_dropdown()  # Initialize the levels dropdown

plot_output = widgets.Output()

def ask_errors(k, M, nsamp, EbN0_db):
    L = 2**k
    SNR_db = EbN0_db - 10*np.log10(nsamp/(2*k))
    SNR = 10 ** (SNR_db * 0.1)
    x = 2 * np.floor(L * np.random.rand(M)) - L + 1
    P_x = (L**2 - 1) / 3
    Measured_x = np.sum(x**2) / len(x)

    y = []
    for i in range(len(x)):
        y.extend([x[i]] * nsamp)
    y = np.array(y)

    noise = np.random.normal(0, np.sqrt(Measured_x / SNR), len(y))
    y_noisy = y + noise

    y = np.reshape(y_noisy, (M, nsamp))
    matched = np.ones((nsamp, 1))
    z = np.matmul(y, matched) / nsamp
    l = np.arange(-L+1, L, 2)

    z = z[:, 0]
    errors = 0
    for i in range(len(z)):
        differences = np.abs(l - z[i])
        m = np.min(differences)
        index = np.where(differences == m)[0][0]
        z[i] = l[index]
        if x[i] != z[i]:
            errors += 1
    
    return errors

# Placeholder error functions
def qam_errors(k, M, nsamp, EbN0_db):
    # Placeholder for QAM error function
    return 0

def psk_errors(k, M, nsamp, EbN0_db):
    # Placeholder for PSK error function
    return 0

def fsk_errors(k, M, nsamp, EbN0_db, coherent=True):
    # Placeholder for FSK error function
    return 0

def msk_errors(M, nsamp, EbN0_db, precoding=True):
    # Placeholder for MSK error function
    return 0

# Update Eb/N0 dB range to go up to 20 dB
M = 20000
nsamp = 16
EbN0_db = np.arange(0, 21, 2)  # Now goes up to 20 dB
EbN0 = 10 ** (EbN0_db / 10)

def plot_selected_modulations(change):
    # Start timer
    start_time = time.time()

    with plot_output:
        loader_html3.value = loading
        plot_output.clear_output(wait=True)
        plt.figure(figsize=(10, 7))
        
        modulation = modulation_dropdown.value
        level = levels_dropdown.value
        if level is None:
            return
        L = int(level.split('-')[0])
        k = int(np.log2(L))
        modulation_name = level
        
        if modulation == 'ASK':
            ber = [ask_errors(k, M, nsamp, db) / M / np.log2(L) for db in EbN0_db]
            ber_theoretical = (((L - 1) / L) * erfc(np.sqrt(EbN0 * (3 * np.log2(L)) / (L**2 - 1)))) / k
        elif modulation == 'QAM':
            ber = [qam_errors(k, M, nsamp, db) / M / np.log2(L) for db in EbN0_db]
            ber_theoretical = erfc(np.sqrt(3 * EbN0 / (2 * (L - 1))))
        elif modulation == 'PSK':
            ber = [psk_errors(k, M, nsamp, db) / M / np.log2(L) for db in EbN0_db]
            ber_theoretical = erfc(np.sqrt(EbN0 * np.sin(np.pi / L)))
        elif modulation == 'FSK':
            ber = [fsk_errors(k, M, nsamp, db, coherent=True) / M / np.log2(L) for db in EbN0_db]
            ber_theoretical = erfc(np.sqrt(EbN0 / (2 * np.log2(L))))
        elif modulation == 'MSK':
            ber = [msk_errors(M, nsamp, db, precoding=True) / M for db in EbN0_db]
            ber_theoretical = erfc(np.sqrt(EbN0))

        plt.semilogy(EbN0_db, ber, 'o', label=f'Experimental {modulation_name}', color='#1F77B4')
        plt.semilogy(EbN0_db, ber_theoretical, linestyle='-', label=f'Theoretical {modulation_name}', color='#00CC96')

        plt.grid(True, which='both')
        plt.xlabel("Eb/N0 (dB)")
        plt.ylabel("BER")
        plt.legend()
        plt.title('Theoretical and Experimental BER')

        # Show elapsed time
        elapsed_time = time.time() - start_time
        timer_html3.value = f"Elapsed time: {elapsed_time:.2f} seconds"
        loader_html3.value = done

        plt.show()

# Attach the update_plot function to the 'value' property of the dropdowns
modulation_dropdown.observe(plot_selected_modulations, names='value')
levels_dropdown.observe(plot_selected_modulations, names='value')

inputs = widgets.VBox([modulation_dropdown, levels_dropdown])

# Group the loader and timer together (they will appear next to each other horizontally)
loader_timer_box = widgets.VBox([loader_html3, timer_html3], layout=widgets.Layout(margin='0 0 0 20px', width='auto'))

ui = widgets.HBox([inputs, loader_timer_box], layout=Layout(align_items='center'))

# Setup the display layout
display(ui, plot_output)

# Display the initial plot
plot_selected_modulations(None)


Output()